In [45]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pickle as cPickle
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, log_loss
import graphviz
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Concatenate, Input
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import cv2
import tensorflow as tf
from PIL import Image

In [8]:
img1 = cv2.imread('db_notehead/score12_trimmed.png')
img1.shape

(231, 612, 3)

We'll resize to (200,600,3).

In [9]:
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection

In [36]:
def getModel_Test():
    model = Sequential([
        BatchNormalization(input_shape=(200, 600, 3)),
        Conv2D(8, 3, 3, activation='relu', name='conv_one'),
        Conv2D(8, 3, 3, activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(225, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [37]:
def preprocess_input(input):
    img = image.load_img(input, target_size=(200,600))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [54]:
train_paths = []

x_pictures=[]
x_segments=[]
f=open('db_notehead/annotationBox.csv')
fr=csv.reader(f,delimiter=";")
print(fr)
for i in fr:
    str1 = i[0]
    slice1 = str1[str1.find("db_"):str1.find(".png")] + ".png"
    if slice1 not in train_paths:
        train_paths.append(slice1)



# for i in fr:

# traingen=ImageDataGenerator(fill_mode="constant")
# #same with y_pictures and y_segments for validation data

In [122]:
x_segments = []

In [123]:
f=open('db_notehead/annotationBox.csv')
fr=csv.reader(f,delimiter=";")

count = 0
tempList = []

for i in fr:
    str1 = i[0]
    slice1 = str1[str1.find(".png,"):str1.find(",notehead")]
    outSlice = slice1[5:]
    outSliceList = [int(x) for x in outSlice.split(',')]
#     print(str1)
#     print("score" + str(count+1)+"_trimmed")
#     print("score" + str(count+1)+"_trimmed" in  str1)
    if "score"+str(count+1)+"_trimmed" in str1:
        tempList.append(np.array([outSliceList[0],outSliceList[1],outSliceList[2],outSliceList[3]]))
    else:
        count += 1
        x_segments.append(tempList)
        tempList = []
        if "score"+str(count+1)+"_trimmed" in str1:
            tempList.append(np.array([outSliceList[0],outSliceList[1],outSliceList[2],outSliceList[3]]))
    #print(tempList)
    #x_segments.append([tempList])

In [125]:
len(x_segments)

29

In [77]:
from keras.preprocessing.image import img_to_array, load_img
for training_picture_path in train_paths:
    img=load_img(training_picture_path)
    x=img_to_array(img)
    x=preprocess(x)
    x_pictures.append(x)

In [82]:
#len(x_pictures)
len(x_segments)

2153

In [126]:
x_pictures

[array([[[64921.062, 64908.223, 64901.32 ],
         [64921.062, 64908.223, 64901.32 ],
         [64921.062, 64908.223, 64901.32 ],
         ...,
         [64921.062, 64908.223, 64901.32 ],
         [64921.062, 64908.223, 64901.32 ],
         [64921.062, 64908.223, 64901.32 ]],
 
        [[64921.062, 64908.223, 64901.32 ],
         [64921.062, 64908.223, 64901.32 ],
         [64921.062, 64908.223, 64901.32 ],
         ...,
         [64921.062, 64908.223, 64901.32 ],
         [64921.062, 64908.223, 64901.32 ],
         [64921.062, 64908.223, 64901.32 ]],
 
        [[64921.062, 64908.223, 64901.32 ],
         [64921.062, 64908.223, 64901.32 ],
         [64921.062, 64908.223, 64901.32 ],
         ...,
         [64921.062, 64908.223, 64901.32 ],
         [64921.062, 64908.223, 64901.32 ],
         [64921.062, 64908.223, 64901.32 ]],
 
        ...,
 
        [[64921.062, 64908.223, 64901.32 ],
         [64921.062, 64908.223, 64901.32 ],
         [64921.062, 64908.223, 64901.32 ],
         .

In [127]:
# make validation from x_pictures and x_segments
y_pictures = x_pictures[20:28]
y_segments = x_segments[20:28]

In [128]:
len(y_pictures)

8

In [129]:
x_pictures_new = x_pictures[:20]
x_segments_new = x_segments[:20]

In [130]:
len(x_segments_new)

20

In [85]:
traingen=ImageDataGenerator(fill_mode="constant")

In [132]:
model.fit_generator(traingen.flow(x=np.array(x_pictures_new), 
    y=np.array(x_segments_new),
    batch_size=4),validation_data=valgen.flow(x=np.array(y_pictures), 
    y=np.array(y_segments)),epochs=15,steps_per_epoch=50,verbose=1)

ValueError: setting an array element with a sequence.